In [2]:
from nequip.ase import NequIPCalculator
from pymatgen.io.ase import AseAtomsAdaptor
from ase.filters import UnitCellFilter 
from ase.constraints import FixAtoms
from ase.io import Trajectory
import pickle
from ase.atoms import Atoms, units 
import numpy as np 
import json, os
from ase.optimize import LBFGS 
from pymatgen.core import Structure 

class TrajectoryObserver:
    """Trajectory observer is a hook in the relaxation process that saves the
    intermediate structures.
    """

    # thanks to CHGNet and M3GNET teams

    def __init__(self, atoms: Atoms) -> None:
        """Create a TrajectoryObserver from an Atoms object.

        Args:
            atoms (Atoms): the structure to observe.
        """
        self.atoms = atoms
        self.energies: list[float] = []
        self.forces: list[np.ndarray] = []
        #self.stresses: list[np.ndarray] = []
        #self.magmoms: list[np.ndarray] = []
        self.atom_positions: list[np.ndarray] = []
        self.cells: list[np.ndarray] = []

    def __call__(self) -> None:
        """The logic for saving the properties of an Atoms during the relaxation."""
        self.energies.append(self.compute_energy())
        self.forces.append(self.atoms.get_forces())
        #self.stresses.append(self.atoms.get_stress())
        #self.magmoms.append(self.atoms.get_magnetic_moments())
        self.atom_positions.append(self.atoms.get_positions())
        self.cells.append(self.atoms.get_cell()[:])

    def __len__(self) -> int:
        """The number of steps in the trajectory."""
        return len(self.energies)

    def compute_energy(self) -> float:
        """Calculate the potential energy.

        Returns:
            energy (float): the potential energy.
        """
        return self.atoms.get_potential_energy()

    def save(self, filename: str) -> None:
        """Save the trajectory to file.

        Args:
            filename (str): filename to save the trajectory
        """
        out_pkl = {
            "energy": self.energies,
            "forces": self.forces,
            #"stresses": self.stresses,
            #"magmoms": self.magmoms,
            "atom_positions": self.atom_positions,
            "cell": self.cells,
            "atomic_number": self.atoms.get_atomic_numbers(),
        }
        with open(filename, "wb") as file:
            pickle.dump(out_pkl, file)


def allegro_relaxer(atoms, potential_path, species, device='cpu', fmax = 0.01, steps = 250, verbose=False, relax_cell=True, loginterval=1):
    atoms.calc = NequIPCalculator.from_deployed_model(
        model_path=potential_path,
        species_to_type_name = species
    )
    
    if relax_cell:
        ucf = UnitCellFilter(atoms)
        obs = TrajectoryObserver(ucf)
        optimizer = LBFGS(ucf)
        optimizer.attach(obs, interval=loginterval)
        
    else:
        constraints = FixAtoms(mask=[False] * len(atoms))  # Allow all atoms to move
        # Add constraints to atoms
        atoms.set_constraint(constraints)

        obs = TrajectoryObserver(atoms)
        optimizer = LBFGS(atoms)
        optimizer.attach(obs, interval=loginterval)
    
    optimizer.run(fmax=fmax, steps=steps)
    struct = AseAtomsAdaptor.get_structure(atoms)
    return {"final_structure" : struct, "trajectory" : obs}

def numeric_stress(atoms, d=1e-6, voigt=True):
    stress = np.zeros((3, 3), dtype=float)

    cell = atoms.cell.copy()
    V = atoms.get_volume()
    for i in range(3):
        x = np.eye(3)
        x[i, i] += d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        eplus = atoms.get_potential_energy(force_consistent=True)

        x[i, i] -= 2 * d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        eminus = atoms.get_potential_energy(force_consistent=True)

        stress[i, i] = (eplus - eminus) / (2 * d * V)
        x[i, i] += d

        j = i - 2
        x[i, j] = d
        x[j, i] = d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        eplus = atoms.get_potential_energy(force_consistent=True)

        x[i, j] = -d
        x[j, i] = -d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        eminus = atoms.get_potential_energy(force_consistent=True)

        stress[i, j] = (eplus - eminus) / (4 * d * V)
        stress[j, i] = stress[i, j]
    atoms.set_cell(cell, scale_atoms=True)

    if voigt:
        return stress.flat[[0, 4, 8, 5, 2, 1]]
    else:
        return stress

def np_numeric_stress(atoms, d=1e-6, voigt=True):
    stress = np.zeros((3, 3), dtype=float)
    cell = atoms.cell.copy()
    V = atoms.get_volume()

    for i in range(3):
        x = np.eye(3)
        x[i, i] += d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        eplus = atoms.get_potential_energy(force_consistent=True)
        
        x[i, i] -= 2 * d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        eminus = atoms.get_potential_energy(force_consistent=True)
        
        stress[i, i] = (eplus - eminus) / (2 * d * V)
        x[i, i] += d
        
        for j in range(i + 1, 3):
            x[i, j] = d
            x[j, i] = d
            atoms.set_cell(np.dot(cell, x), scale_atoms=True)
            eplus = atoms.get_potential_energy(force_consistent=True)
            
            x[i, j] = -d
            x[j, i] = -d
            atoms.set_cell(np.dot(cell, x), scale_atoms=True)
            eminus = atoms.get_potential_energy(force_consistent=True)
            
            stress[i, j] = (eplus - eminus) / (4 * d * V)
            stress[j, i] = stress[i, j]
    
    atoms.set_cell(cell, scale_atoms=True)

    return stress

/opt/homebrew/Caskroom/miniforge/base/envs/allegro/lib/python3.10/site-packages/nequip/__init__.py:20: UserWarning: !! PyTorch version 1.13.1 found. Upstream issues in PyTorch versions 1.13.* and 2.* have been seen to cause unusual performance degredations on some CUDA systems that become worse over time; see https://github.com/mir-group/nequip/discussions/311. The best tested PyTorch version to use with CUDA devices is 1.11; while using other versions if you observe this problem, an unexpected lack of this problem, or other strange behavior, please post in the linked GitHub issue.
  warnings.warn(


In [3]:
# test the relaxing function
#load all the structures
# on pc
data_path = '../Visualization/Job_Structures/Post_VASP/VCrTiWZr_Summit/gen_0_4/Vacancies/converted_vcrtiwzr_gen_0_4_vacancies_data.json'
# on mac 
data_path = '../Visualization/Job_Structures/Post_VASP/VCrTiWZr_Summit/Vacancies/converted_vcrtiwzr_gen_0_4_vacancies_data.json'
data = json.load(open(data_path, 'r'))

In [4]:
species = {
            "Ti": "NequIPTypeNameForTitanium",
            "V": "NequIPTypeNameForVanadium",
            "Cr" : "NequIPTypeNameForChromium",
            "Zr" : "NequIPTypeNameForZirconium",
            "W" : "NequIPTypeNameForTungsten",
        }

species = {"Ti" : "Ti", "V" : "V", "Cr" : "Cr", "Zr" : "Zr", "W" : "W"}

In [5]:
# get a structure 
structure = Structure.from_dict(data['supercell_gen0_comp11_struct3_vac_site3_start']['structures'][-1])
test_energy = data['supercell_gen0_comp11_struct3_vac_site3_start']['energies'][-1]
test_forces = data['supercell_gen0_comp11_struct3_vac_site3_start']['forces'][-1] 
test_stress = np.array(data['supercell_gen0_comp11_struct3_vac_site3_start']['stresses'][-1])

atoms = AseAtomsAdaptor.get_atoms(structure)

In [20]:
from pymatgen.core import Structure, Lattice
from pymatgen.io.vasp.inputs import Poscar 

# Lattice parameter
a = 3.01

# Rhombohedral lattice vectors for the primitive cell of a BCC lattice
lattice_vectors = [
    [0.5 * a, 0.5 * a, -0.5 * a],
    [0.5 * a, -0.5 * a, 0.5 * a],
    [-0.5 * a, 0.5 * a, 0.5 * a],
]

# Create the lattice
lattice = Lattice(lattice_vectors)

# Define the species and position for the single Vanadium atom in the primitive cell
species = ["V"]
position = [0, 0, 0]  # At the origin

# Create the structure
structure = Structure(lattice, species, [position])
for i in range(3,8):
    supercell = structure.copy()
    supercell.make_supercell([i, i, i])
    poscar = Poscar(supercell)
    poscar.write_file(f"new_POSCAR_{i*i*i}_atoms")


In [6]:
print(test_stress)

[[-0.02959  0.02115 -0.00338]
 [ 0.02115 -0.00875  0.01259]
 [-0.00338  0.01259 -0.00939]]


In [10]:
kb_to_ev_ang_3 = 6.242e-4
test_virial = -1 * test_stress * kb_to_ev_ang_3 * structure.volume
print(test_virial)
print(test_virial.tolist())

[[ 0.01634    -0.01167931  0.00186648]
 [-0.01167931  0.00483187 -0.00695237]
 [ 0.00186648 -0.00695237  0.00518528]]
[[0.016339995297547327, -0.011679313975773096, 0.0018664813824166937], [-0.011679313975773096, 0.004831867484066885, -0.006952367042788808], [0.0018664813824166937, -0.006952367042788808, 0.0051852840771872055]]


In [13]:
pot_path = '../Potentials/vcrtiwzr_vac_deployed.pth'
#relax_endpoint = allegro_relaxer(atoms, potential_path= pot_path , species = species , relax_cell= False)
atoms.calc = NequIPCalculator.from_deployed_model(
        model_path=pot_path,
        species_to_type_name = species
    )

print(atoms.get_potential_energy())


/opt/homebrew/Caskroom/miniforge/base/envs/allegro/lib/python3.10/site-packages/nequip/utils/_global_options.py:62: UserWarning: Setting the GLOBAL value for jit fusion strategy to `[('DYNAMIC', 3)]` which is different than the previous value of `[('STATIC', 2), ('DYNAMIC', 10)]`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/allegro/lib/python3.10/site-packages/nequip/utils/_global_options.py:108: UserWarning: Setting the GLOBAL value for torch.set_default_dtype to `torch.float64` which is different than the previous value of `torch.float32`
  warnings.warn(


-583.4518974178338


In [ ]:
nums = np.logspace(-7, 2, num=9)
for num in nums:
    stress = numeric_stress(atoms, d=num, voigt=False)
    #print(stress)
    #print(np.linalg.norm(stress - test_stress))
    mag_dif = np.abs(stress - test_stress)
    print("Step = ", num)
    print(mag_dif)
    #print("\n")
    

In [72]:
# benchmark stress and np stress
import time
import numpy as np

# Assuming `atoms` is already defined and has a calculator set as shown previously
# Define or import your functions `numeric_stress` and `np_numeric_stress` as needed

# Benchmark numeric_stress
start_time = time.time()
stress_tensor_numeric = numeric_stress(atoms, voigt=False)
time_numeric = time.time() - start_time

# Benchmark np_numeric_stress
start_time = time.time()
stress_tensor_np = np_numeric_stress(atoms)
time_np = time.time() - start_time

# Calculate the difference between the stress tensors
stress_difference = np.abs(stress_tensor_numeric - stress_tensor_np)

# Print the results
print(f"Time for numeric_stress: {time_numeric:.4f} seconds")
print(f"Time for np_numeric_stress: {time_np:.4f} seconds")
print(f"Difference between stress tensors: {stress_difference}")

Time for numeric_stress: 2.0849 seconds
Time for np_numeric_stress: 1.9206 seconds
Difference between stress tensors: [[0.         0.         0.00026443]
 [0.         0.         0.        ]
 [0.00026443 0.         0.        ]]


In [ ]:
#print(atoms.get_forces())

# print the volume of the cell
print(atoms.get_volume())

from ase.constraints import FixAtoms
from ase.filters import UnitCellFilter
# Apply constraints to fix the cell
constraints = FixAtoms(mask=[False] * len(atoms))  # Allow all atoms to move

# Add constraints to atoms
atoms.set_constraint(constraints)

# Create a UnitCellFilter, but since the cell is fixed, it only affects atomic positions
#ucf = UnitCellFilter(atoms, mask=[False, False, False])

# Perform the relaxation
opt = LBFGS(atoms)
opt.run(fmax=0.01)  # Convergence criterion for forces in eV/Å

# Output the relaxed structure
print(atoms.get_volume())

In [59]:
relaxed = allegro_relaxer(atoms, potential_path= pot_path , species = species , relax_cell= False)

       Step     Time          Energy          fmax
LBFGS:    0 16:16:14     -583.451897        0.028008
LBFGS:    1 16:16:15     -583.452094        0.024919
LBFGS:    2 16:16:15     -583.453019        0.030159
LBFGS:    3 16:16:15     -583.453073        0.026159
LBFGS:    4 16:16:15     -583.453285        0.011257
LBFGS:    5 16:16:15     -583.453311        0.015394
LBFGS:    6 16:16:16     -583.453376        0.017373
LBFGS:    7 16:16:16     -583.453410        0.011784
LBFGS:    8 16:16:16     -583.453436        0.007593


In [32]:
#from ase.calculators.test import numeric_stress 
print(numeric_stress(atoms))

884.6738653484483
[-0.00396188  0.00032333 -0.00020627  0.00108414 -0.00030494 -0.00027663]


In [33]:
comp_stress = numeric_stress(atoms, voigt=False)
print(comp_stress)  

884.6738653484483
[[-0.00396188 -0.00027663 -0.00030494]
 [-0.00027663  0.00032333  0.00108414]
 [-0.00030494  0.00108414 -0.00020627]]


In [42]:
#print(test_stress)
#multiplied_list = [[element / 10 for element in sublist] for sublist in test_stress]
difference = np.array(test_stress)/10 - np.array(comp_stress)
print(np.array(test_stress)/10)
print(np.array(comp_stress))
print(difference)
#for i in range(3): 
    #for j in range(3):
        

[[-0.002959  0.002115 -0.000338]
 [ 0.002115 -0.000875  0.001259]
 [-0.000338  0.001259 -0.000939]]
[[-0.00396188 -0.00027663 -0.00030494]
 [-0.00027663  0.00032333  0.00108414]
 [-0.00030494  0.00108414 -0.00020627]]
[[ 1.00287768e-03  2.39163236e-03 -3.30581433e-05]
 [ 2.39163236e-03 -1.19832942e-03  1.74863358e-04]
 [-3.30581433e-05  1.74863358e-04 -7.32731840e-04]]


In [37]:
np_stress = np.array(test_stress)
print(np_stress)
print(np_stress*10)

[[-0.02959  0.02115 -0.00338]
 [ 0.02115 -0.00875  0.01259]
 [-0.00338  0.01259 -0.00939]]
[[-0.2959  0.2115 -0.0338]
 [ 0.2115 -0.0875  0.1259]
 [-0.0338  0.1259 -0.0939]]


In [25]:
# need to create vasp job for the relaxed structure 
initial_structure = Structure.from_dict(data['supercell_gen0_comp11_struct3_vac_site3_start']['structures'][0])

# make a POSCAR file for this 
from pymatgen.io.vasp.inputs import Poscar
poscar = Poscar(initial_structure)
poscar.write_file('test_stress_POSCAR')

In [ ]:
vcrti_data = json.load(open('../Visualization/Job_Structures/Post_VASP/VCrTi/shuffled_vacancies_data.json'))
